In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt

# EPOCH_FILE = "measure-2025-11-27_15:19:46-epo.fif"
EPOCH_FILE = "measure-2025-11-27_15:17:48-epo-H.fif"
epochs = mne.read_epochs(EPOCH_FILE, preload=True)

# Apply bandpass filter 1 - 40 Hz
epochs.filter(l_freq=1, h_freq=40.0, fir_design='firwin')

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class MaxMinScaler(BaseEstimator, TransformerMixin):
    """Min-Max Scaler for normalizing data.
    Normalization occurs per channel.
    """

    def __init__(self, feature_range: tuple = (-1, 1), eps: float = 1e-8):
        """Min-Max Scaler for 3D arrays.

        Args:
            feature_range (tuple, optional): Desired range of transformed data. Defaults to (-1, 1).
            eps (float, optional): Small value to avoid division by zero. Defaults to 1e-8.
        """
        self.feature_range = feature_range
        self.eps = eps

    def fit(self, X: np.ndarray, y=None):
        # Calculate min and max values for each channel
        self._min = X.min(axis=(0, 2)).reshape((1, -1, 1))
        self._max = X.max(axis=(0, 2)).reshape((1, -1, 1))
        return self

    def transform(self, X: np.ndarray, y=None):
        denom = self._max - self._min
        denom = np.where(denom < self.eps, 1, denom)  # Prevent division by zero
        X = (X - self._min) / denom
        X = X * (self.feature_range[1] - self.feature_range[0]) + self.feature_range[0]
        return X

data = epochs.get_data(copy=False)
scaler = MaxMinScaler(feature_range=(-50, 50))
data = scaler.fit_transform(data)
epochs._data = data
print(f"Max value: {epochs.get_data(copy=True).max():.2e}")
print(f"Min value: {epochs.get_data(copy=True).min():.2e}")


In [ ]:
# Plot raw data for all channels using MNE's built-in method
# Crop to 20-40 seconds time window
epochs[0].copy().crop(tmin=50, tmax=100).plot(scalings=dict(eeg=40, grad=40, mag=40))

In [ ]:




# epochs.plot(scalings="auto", show=True)
X = epochs.get_data()  # shape (n_epochs, n_channels, n_times)
first_channel_data = X[:, 1, 2000:20000]  # Data for the first channel across all epochs

# l_freq, h_freq = 0.5, 40.0
# first_channel_data = mne.filter.filter_data(first_channel_data, 500, l_freq, h_freq, fir_design='firwin')



n_epochs, n_times = first_channel_data.shape
print(f"{n_epochs = }")
times = np.arange(n_times) / float(epochs.info['sfreq'])

plt.figure(figsize=(15, 6))
for i in range(1):
    plt.plot(times, first_channel_data[i], alpha=0.7, label=f'epoch {i}' if n_epochs <= 10 else None)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.title(f'First channel: {epochs.ch_names[0]}')
if n_epochs <= 10:
    plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
bands = {
    "alpha": (8, 13),
    "beta": (13, 30),
    "gamma": (30, 45),
}

In [ ]:
# Compute band power for alpha, beta, and gamma for each channel and epoch
from mne.time_frequency import psd_array_welch

# Define frequency bands
bands = {
    "alpha": (8, 13),
    "beta": (13, 30),
    "gamma": (30, 40),  # Limited to 40 Hz due to filter
}

# Get data and sampling frequency
X = epochs.get_data()  # shape: (n_epochs, n_channels, n_times)
sfreq = epochs.info['sfreq']
n_epochs, n_channels, n_times = X.shape

# Compute PSD for each epoch and channel
band_powers = {band: np.zeros((n_epochs, n_channels)) for band in bands.keys()}

for ep_idx in range(n_epochs):
    for ch_idx in range(n_channels):
        # Compute PSD using Welch's method
        psds, freqs = psd_array_welch(
            X[ep_idx, ch_idx, :], 
            sfreq=sfreq, 
            fmin=0.5, 
            fmax=40.0,
            n_fft=int(sfreq * 2),  # 2-second windows
            n_overlap=int(sfreq)    # 1-second overlap
        )
        
        # Compute band power for each frequency band
        for band_name, (fmin, fmax) in bands.items():
            freq_mask = (freqs >= fmin) & (freqs <= fmax)
            band_powers[band_name][ep_idx, ch_idx] = np.mean(psds[freq_mask])

# Plot band powers for each channel and epoch
event_names = list(epochs.event_id.keys())
fig, axes = plt.subplots(n_channels, 1, figsize=(12, 4 * n_channels))
if n_channels == 1:
    axes = [axes]

x_pos = np.arange(len(bands))
width = 0.35

for ch_idx, ax in enumerate(axes):
    for ep_idx in range(n_epochs):
        event_name = event_names[ep_idx] if ep_idx < len(event_names) else f'Epoch {ep_idx}'
        powers = [band_powers[band][ep_idx, ch_idx] for band in bands.keys()]
        
        offset = width * (ep_idx - n_epochs/2 + 0.5)
        ax.bar(x_pos + offset, powers, width, label=event_name, alpha=0.8)
    
    ax.set_ylabel('Power')
    ax.set_title(f'Channel: {epochs.ch_names[ch_idx]}')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(bands.keys())
    ax.legend()
    ax.grid(True, alpha=0.3)

fig.suptitle('Band Power (Alpha, Beta, Gamma) per Channel and Epoch', fontsize=14, y=1.0)
plt.tight_layout()
plt.show()

# Print numerical values
print("\nBand Power Values:")
for band_name in bands.keys():
    print(f"\n{band_name.upper()} ({bands[band_name][0]}-{bands[band_name][1]} Hz):")
    for ep_idx in range(n_epochs):
        event_name = event_names[ep_idx] if ep_idx < len(event_names) else f'Epoch {ep_idx}'
        print(f"  {event_name}:")
        for ch_idx in range(n_channels):
            print(f"    {epochs.ch_names[ch_idx]}: {band_powers[band_name][ep_idx, ch_idx]:.2e}")

In [ ]:
spectrum =  epochs[0].compute_psd()
spectrum.plot_topomap()

In [ ]:
epochs.plot_sensors(kind="topomap", ch_type="all")
epochs.ch_names

In [ ]:
# Alpha Band Hemispheric Asymmetry Analysis for Depression Detection
# Based on: MDD patients show Right < Left in frontal, and Left < Right in temporal/parietal/occipital
from mne.time_frequency import psd_array_welch

# Define channel pairs (left vs right hemisphere)
# Frontal: Fp1 (left) vs Fp2 (right)
# Temporal: T3 (left) vs T4 (right), T5 (left) vs T6 (right)
# Central: Cz (midline - reference)
# Occipital: Oz (midline - reference)

channel_pairs = {
    'Frontal': ('Fp1', 'Fp2'),
    'Temporal (anterior)': ('T3', 'T4'),
    'Temporal (posterior)': ('T5', 'T6')
}

# Alpha band definition
alpha_band = (8, 13)

# Get data
X = epochs.get_data()
sfreq = epochs.info['sfreq']
n_epochs = X.shape[0]
event_names = list(epochs.event_id.keys())

# Compute alpha PSD for each channel
alpha_psd = {}
for ch_name in epochs.ch_names:
    ch_idx = epochs.ch_names.index(ch_name)
    alpha_psd[ch_name] = []
    
    for ep_idx in range(n_epochs):
        psds, freqs = psd_array_welch(
            X[ep_idx, ch_idx, :],
            sfreq=sfreq,
            fmin=alpha_band[0],
            fmax=alpha_band[1],
            n_fft=int(sfreq * 2),
            n_overlap=int(sfreq)
        )
        alpha_psd[ch_name].append(np.mean(psds))

# Create comparison plots
fig, axes = plt.subplots(len(channel_pairs), 1, figsize=(12, 5 * len(channel_pairs)))
if len(channel_pairs) == 1:
    axes = [axes]

for idx, (region, (left_ch, right_ch)) in enumerate(channel_pairs.items()):
    ax = axes[idx]
    
    x_pos = np.arange(n_epochs)
    width = 0.35
    
    left_powers = alpha_psd[left_ch]
    right_powers = alpha_psd[right_ch]
    
    bars1 = ax.bar(x_pos - width/2, left_powers, width, label=f'{left_ch} (Left)', alpha=0.8, color='blue')
    bars2 = ax.bar(x_pos + width/2, right_powers, width, label=f'{right_ch} (Right)', alpha=0.8, color='red')
    
    # Add asymmetry ratio annotations
    for ep_idx in range(n_epochs):
        ratio = left_powers[ep_idx] / right_powers[ep_idx] if right_powers[ep_idx] != 0 else 0
        event_name = event_names[ep_idx] if ep_idx < len(event_names) else f'Epoch {ep_idx}'
        
        # Determine if pattern matches depression (for frontal: Right < Left means ratio > 1)
        if region == 'Frontal':
            depression_indicator = "⚠️ MDD pattern" if ratio > 1.0 else "✓ Healthy pattern"
        else:  # Temporal/Parietal/Occipital
            depression_indicator = "⚠️ MDD pattern" if ratio < 1.0 else "✓ Healthy pattern"
        
        ax.text(ep_idx, max(left_powers[ep_idx], right_powers[ep_idx]) * 1.05,
                f'{event_name}\\nL/R ratio: {ratio:.2f}\\n{depression_indicator}',
                ha='center', va='bottom', fontsize=9)
    
    ax.set_ylabel('Alpha Band Power (8-13 Hz)')
    ax.set_title(f'{region} Region - Alpha Asymmetry', fontsize=12, fontweight='bold')
    ax.set_xticks(x_pos)
    ax.set_xticklabels([event_names[i] if i < len(event_names) else f'Epoch {i}' for i in range(n_epochs)])
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')

fig.suptitle('EEG Alpha Inter-hemispheric Asymmetry for Depression Assessment', 
             fontsize=14, fontweight='bold', y=1.0)
plt.tight_layout()
plt.show()

# Print detailed analysis
print("="*70)
print("ALPHA BAND HEMISPHERIC ASYMMETRY ANALYSIS")
print("="*70)
print("\\nReference: MDD Patients show:")
print("  - Frontal: Right < Left (higher left frontal alpha)")
print("  - Temporal/Parietal/Occipital: Left < Right (higher right alpha)")
print("="*70)

for region, (left_ch, right_ch) in channel_pairs.items():
    print(f"\\n{region.upper()} REGION ({left_ch} vs {right_ch}):")
    print("-" * 70)
    
    for ep_idx in range(n_epochs):
        event_name = event_names[ep_idx] if ep_idx < len(event_names) else f'Epoch {ep_idx}'
        left_power = alpha_psd[left_ch][ep_idx]
        right_power = alpha_psd[right_ch][ep_idx]
        ratio = left_power / right_power if right_power != 0 else 0
        
        print(f"\\n  {event_name}:")
        print(f"    {left_ch} (Left):  {left_power:.4f}")
        print(f"    {right_ch} (Right): {right_power:.4f}")
        print(f"    L/R Ratio: {ratio:.4f}")
        
        if region == 'Frontal':
            if ratio > 1.0:
                print(f"    ⚠️  MATCHES MDD PATTERN (Left > Right)")
            else:
                print(f"    ✓  Matches healthy pattern (Right >= Left)")
        else:
            if ratio < 1.0:
                print(f"    ⚠️  MATCHES MDD PATTERN (Left < Right)")
            else:
                print(f"    ✓  Matches healthy pattern (Left >= Right)")

print("\\n" + "="*70)
print("OVERALL ASSESSMENT:")
print("="*70)
mdd_indicators = 0
total_checks = len(channel_pairs) * n_epochs

for region, (left_ch, right_ch) in channel_pairs.items():
    for ep_idx in range(n_epochs):
        ratio = alpha_psd[left_ch][ep_idx] / alpha_psd[right_ch][ep_idx]
        if (region == 'Frontal' and ratio > 1.0) or (region != 'Frontal' and ratio < 1.0):
            mdd_indicators += 1

print(f"Depression indicators found: {mdd_indicators}/{total_checks}")
print(f"Percentage: {(mdd_indicators/total_checks)*100:.1f}%")
print("\\nNote: This is a simplified analysis. Clinical diagnosis requires")
print("comprehensive evaluation by qualified healthcare professionals.")

In [ ]:
# Band Power Over Time Analysis
# Part 1: Spectrograms for each epoch (averaged across channels)
# Part 2: Band comparison plots (alpha, beta, gamma) showing both epochs
from scipy.signal import spectrogram

# Define frequency bands
bands = {
    "theta": (4, 8),
    "alpha": (8, 13),
    "beta": (13, 30),
    "gamma": (30, 40),
}

# Get data
X = epochs.get_data()  # shape: (n_epochs, n_channels, n_times)
sfreq = epochs.info['sfreq']
n_epochs, n_channels, n_times = X.shape
event_names = list(epochs.event_id.keys())

# Storage for band powers over time for all epochs
all_band_powers = {band: [] for band in bands.keys()}
time_axis = None

# ============================================================================
# PART 1: SPECTROGRAMS - One per epoch (averaged across channels)
# ============================================================================
print("="*70)
print("PART 1: SPECTROGRAMS (averaged across all channels)")
print("="*70)

for ep_idx in range(n_epochs):
    event_name = event_names[ep_idx] if ep_idx < len(event_names) else f'Epoch {ep_idx}'
    print(f"Processing spectrogram for {event_name}...")
    
    # Average spectrogram across all channels
    Sxx_avg = None
    
    for ch_idx in range(n_channels):
        f, t, Sxx = spectrogram(
            X[ep_idx, ch_idx, :],
            fs=sfreq,
            nperseg=int(sfreq * 4),  # 4-second windows
            noverlap=int(sfreq * 3)  # 3-second overlap
        )
        
        if Sxx_avg is None:
            Sxx_avg = Sxx
            if time_axis is None:
                time_axis = t
        else:
            Sxx_avg += Sxx
    
    # Average across channels
    Sxx_avg /= n_channels
    
    # Plot spectrogram
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # Convert to dB scale
    Sxx_db = 10 * np.log10(Sxx_avg + 1e-12)
    
    im = ax.pcolormesh(t, f, Sxx_db, shading='auto', cmap='viridis')
    ax.set_ylim(0, 40)  # Focus on 0-40 Hz
    ax.set_xlabel('Time (s)', fontsize=12)
    ax.set_ylabel('Frequency (Hz)', fontsize=12)
    ax.set_title(f'Spectrogram - {event_name} (averaged across all channels)', 
                 fontsize=14, fontweight='bold')
    
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label('Power (dB)', fontsize=11)
    
    # Add band regions as horizontal lines
    for band_name, (fmin, fmax) in bands.items():
        ax.axhline(fmin, color='white', linestyle='--', alpha=0.5, linewidth=1)
        ax.axhline(fmax, color='white', linestyle='--', alpha=0.5, linewidth=1)
        ax.text(t[-1] * 0.98, (fmin + fmax) / 2, band_name.capitalize(), 
                color='white', fontsize=10, ha='right', va='center',
                bbox=dict(boxstyle='round', facecolor='black', alpha=0.5))
    
    plt.tight_layout()
    plt.show()

# ============================================================================
# PART 2: BAND POWER COMPARISON - One plot per band, comparing epochs
# ============================================================================
print("\\n" + "="*70)
print("PART 2: BAND POWER COMPARISON OVER TIME")
print("="*70)

# Compute band powers over time for all epochs
for ep_idx in range(n_epochs):
    event_name = event_names[ep_idx] if ep_idx < len(event_names) else f'Epoch {ep_idx}'
    print(f"Computing band powers for {event_name}...")
    
    # Initialize storage for this epoch
    band_powers_this_epoch = {band: np.zeros(len(time_axis)) for band in bands.keys()}
    
    # Compute for each channel and average
    for ch_idx in range(n_channels):
        f, t, Sxx = spectrogram(
            X[ep_idx, ch_idx, :],
            fs=sfreq,
            nperseg=int(sfreq * 4),
            noverlap=int(sfreq * 3)
        )
        
        # Extract band power over time
        for band_name, (fmin, fmax) in bands.items():
            freq_mask = (f >= fmin) & (f <= fmax)
            band_power_time = np.mean(Sxx[freq_mask, :], axis=0)
            band_powers_this_epoch[band_name] += band_power_time
    
    # Average across channels
    for band in bands.keys():
        band_powers_this_epoch[band] /= n_channels
        all_band_powers[band].append(band_powers_this_epoch[band])

# Create comparison plots - one per band
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']

for band_name, (fmin, fmax) in bands.items():
    fig, ax = plt.subplots(figsize=(14, 6))
    
    # Plot each epoch
    for ep_idx in range(n_epochs):
        event_name = event_names[ep_idx] if ep_idx < len(event_names) else f'Epoch {ep_idx}'
        ax.plot(
            time_axis,
            all_band_powers[band_name][ep_idx],
            label=event_name,
            color=colors[ep_idx % len(colors)],
            linewidth=2.5,
            alpha=0.8
        )
    
    ax.set_xlabel('Time (s)', fontsize=12)
    ax.set_ylabel('Power (averaged across channels)', fontsize=12)
    ax.set_title(f'{band_name.capitalize()} Band ({fmin}-{fmax} Hz) - Epoch Comparison', 
                 fontsize=14, fontweight='bold')
    ax.legend(loc='upper right', fontsize=12)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics comparison
    print(f"\\n{band_name.upper()} BAND ({fmin}-{fmax} Hz):")
    print("-" * 70)
    for ep_idx in range(n_epochs):
        event_name = event_names[ep_idx] if ep_idx < len(event_names) else f'Epoch {ep_idx}'
        powers = all_band_powers[band_name][ep_idx]
        print(f"  {event_name:12s}: mean={np.mean(powers):.4f}, std={np.std(powers):.4f}, "
              f"min={np.min(powers):.4f}, max={np.max(powers):.4f}")

print("\\n" + "="*70)
print("Analysis complete!")
print("="*70)

In [ ]:
# Spectral Edge Frequency (SEF90) Computation
# SEF90 is the frequency below which 90% of the total spectral power resides
from mne.time_frequency import psd_array_welch

# Get data
X = epochs.get_data()  # shape: (n_epochs, n_channels, n_times)
sfreq = epochs.info['sfreq']
n_epochs, n_channels, n_times = X.shape
event_names = list(epochs.event_id.keys())

print("="*70)
print("SPECTRAL EDGE FREQUENCY (SEF90) ANALYSIS")
print("="*70)
print("SEF90 = Frequency below which 90% of total spectral power resides")
print("="*70)

# Storage for SEF90 values
sef90_results = {
    'per_channel': {},  # SEF90 for each channel in each epoch
    'averaged': []      # SEF90 averaged across all channels for each epoch
}

# Compute SEF90 for each epoch
for ep_idx in range(n_epochs):
    event_name = event_names[ep_idx] if ep_idx < len(event_names) else f'Epoch {ep_idx}'
    print(f"\n{event_name.upper()}:")
    print("-" * 70)
    
    sef90_per_channel = []
    
    # Compute SEF90 for each channel
    for ch_idx in range(n_channels):
        ch_name = epochs.ch_names[ch_idx]
        
        # Compute PSD using Welch's method
        psds, freqs = psd_array_welch(
            X[ep_idx, ch_idx, :],
            sfreq=sfreq,
            fmin=1.0,
            fmax=40.0,
            n_fft=int(sfreq * 4),  # 4-second windows for better frequency resolution
            n_overlap=int(sfreq * 2)
        )
        
        # Compute cumulative power
        total_power = np.sum(psds)
        cumulative_power = np.cumsum(psds)
        cumulative_power_normalized = cumulative_power / total_power
        
        # Find frequency where cumulative power reaches 90%
        idx_90 = np.where(cumulative_power_normalized >= 0.90)[0]
        if len(idx_90) > 0:
            sef90 = freqs[idx_90[0]]
        else:
            sef90 = freqs[-1]  # If 90% not reached, use max frequency
        
        sef90_per_channel.append(sef90)
        
        # Store in dictionary
        if ch_name not in sef90_results['per_channel']:
            sef90_results['per_channel'][ch_name] = []
        sef90_results['per_channel'][ch_name].append(sef90)
        
        print(f"  {ch_name:8s}: SEF90 = {sef90:6.2f} Hz")
    
    # Compute average SEF90 across all channels
    avg_sef90 = np.mean(sef90_per_channel)
    std_sef90 = np.std(sef90_per_channel)
    sef90_results['averaged'].append(avg_sef90)
    
    print(f"\n  Average across all channels: {avg_sef90:.2f} ± {std_sef90:.2f} Hz")

# ============================================================================
# Visualization: Bar plot comparing SEF90 across epochs
# ============================================================================
print("\n" + "="*70)
print("VISUALIZATION")
print("="*70)

# Plot 1: Average SEF90 comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Left plot: Average SEF90 per epoch
x_pos = np.arange(n_epochs)
avg_sef90_values = sef90_results['averaged']
colors_epochs = ['#1f77b4', '#ff7f0e']

bars = ax1.bar(x_pos, avg_sef90_values, color=colors_epochs, alpha=0.8, edgecolor='black', linewidth=1.5)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('SEF90 (Hz)', fontsize=12)
ax1.set_title('Average SEF90 Across All Channels', fontsize=14, fontweight='bold')
ax1.set_xticks(x_pos)
ax1.set_xticklabels([event_names[i] if i < len(event_names) else f'Epoch {i}' for i in range(n_epochs)])
ax1.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for i, (bar, val) in enumerate(zip(bars, avg_sef90_values)):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
             f'{val:.2f} Hz', ha='center', va='bottom', fontsize=11, fontweight='bold')

# Right plot: SEF90 per channel for each epoch
width = 0.35
x_pos_channels = np.arange(n_channels)

for ep_idx in range(n_epochs):
    event_name = event_names[ep_idx] if ep_idx < len(event_names) else f'Epoch {ep_idx}'
    sef90_values = [sef90_results['per_channel'][ch][ep_idx] for ch in epochs.ch_names]
    
    offset = width * (ep_idx - n_epochs/2 + 0.5)
    ax2.bar(x_pos_channels + offset, sef90_values, width, 
            label=event_name, alpha=0.8, color=colors_epochs[ep_idx])

ax2.set_xlabel('Channel', fontsize=12)
ax2.set_ylabel('SEF90 (Hz)', fontsize=12)
ax2.set_title('SEF90 Per Channel', fontsize=14, fontweight='bold')
ax2.set_xticks(x_pos_channels)
ax2.set_xticklabels(epochs.ch_names, rotation=45, ha='right')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# ============================================================================
# Summary and Interpretation
# ============================================================================
print("\n" + "="*70)
print("SUMMARY")
print("="*70)

# Compare epochs
if n_epochs == 2:
    diff = sef90_results['averaged'][1] - sef90_results['averaged'][0]
    percent_change = (diff / sef90_results['averaged'][0]) * 100
    
    print(f"\nSEF90 Comparison:")
    print(f"  {event_names[0]}: {sef90_results['averaged'][0]:.2f} Hz")
    print(f"  {event_names[1]}: {sef90_results['averaged'][1]:.2f} Hz")
    print(f"  Difference: {diff:+.2f} Hz ({percent_change:+.1f}%)")
    
    if abs(diff) < 1.0:
        print("\n  → Similar spectral distribution between epochs")
    elif diff > 0:
        print(f"\n  → {event_names[1]} shows higher SEF90 (more high-frequency activity)")
    else:
        print(f"\n  → {event_names[0]} shows higher SEF90 (more high-frequency activity)")

print("\n" + "="*70)
print("Clinical Note:")
print("  - Lower SEF90 suggests more power in lower frequencies (e.g., delta, theta)")
print("  - Higher SEF90 suggests more power in higher frequencies (e.g., beta, gamma)")
print("  - SEF90 is used in anesthesia monitoring and neurological assessments")
print("="*70)